# Advent of Code 2016
2016-12-28 - Jeroen van Erp

## Preamble
First let's set up some common functions and imports

In [51]:
import re
from heapq import heappush, heappop
from collections import namedtuple, defaultdict, Counter
from hashlib import md5
from bitarray import bitarray
import math
from functools import partial
from itertools import permutations, combinations, islice
import numpy as np
import networkx as nx
import urllib

def Input(day):
    "Open this day's input file."
    filename = 'day{}.in'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("https://raw.githubusercontent.com/hierynomus/code-challenges/master/2016-adventofcode.com/" + filename).text

    
def neighbours4(point):
    """All horizontal and vertical neighbours of a point in a grid"""
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        yield (point[0] + dx, point[1] + dy)

def neighbours8(point):
    """All neighbours (including diagonals) of a point in a grid"""
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 and dy != 0:
                yield (point[0] + dx, point[1] + dy)
                
def manhattan_distance(p1, p2):
    """Return the Manhattan distance between two points"""
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
                

## Day 1: No Time for a Taxicab

### Part 1
Santa's sleigh uses a very high-precision clock to guide its movements, and the clock's oscillator is regulated by stars. Unfortunately, the stars have been stolen... by the Easter Bunny. To save Christmas, Santa needs you to retrieve all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You're airdropped near **Easter Bunny Headquarters** in a city somewhere. "Near", unfortunately, is as close as you can get - the instructions on the Easter Bunny Recruiting Document the Elves intercepted start here, and nobody had time to work them out further.

The Document indicates that you should start at the given coordinates (where you just landed) and face North. Then, follow the provided sequence: either turn left (L) or right (R) 90 degrees, then walk forward the given number of blocks, ending at a new intersection.

There's no time to follow such ridiculous instructions on foot, though, so you take a moment and work out the destination. Given that you can only walk on the street grid of the city, how far is the shortest path to the destination?

For example:

- Following `R2, L3` leaves you 2 blocks East and 3 blocks North, or 5 blocks away.
- `R2, R2, R2` leaves you 2 blocks due South of your starting position, which is 2 blocks away.
- `R5, L5, R5, R3` leaves you 12 blocks away.

**How many blocks away** is Easter Bunny HQ?

In [52]:
# 0 = North, 1 = West, 2 = South, 3 = East
dirs = {0: (0, 1), 1: (1, 0), 2: (0, -1), 3: (-1, 0)}


def walk(point, direction, instruction):
    x, y = point
    direction = turn(direction, instruction[0])
    dx, dy = dirs[direction]
    for _ in range(int(instruction[1:])):
        x, y = x + dx, y + dy
        yield (x, y), direction

def turn(direction, c):
    return (direction + (1 if c == 'R' else -1)) % 4

with Input(1) as f:
    direction = 0
    position = (0, 0)
    for instruction in f.readline().split(', '):
        position, direction = list(walk(position, direction, instruction))[-1]

    print("Day 1.1: {}".format(manhattan_distance((0, 0), position)))

Day 1.1: 287


### Part 2
Then, you notice the instructions continue on the back of the Recruiting Document. Easter Bunny HQ is actually at the first location you visit twice.

For example, if your instructions are `R8, R4, R4, R8`, the first location you visit twice is 4 blocks away, due East.

How many blocks away is the **first location you visit twice**?

In [53]:
with Input(1) as f:
    def solve():
        direction = 0
        position = (0, 0)
        visited = set()
        for instruction in f.readline().split(', '):
            for p, d in walk(position, direction, instruction):
                if p in visited:
                    yield p
                visited.add(p)
                position, direction = p, d
    print("Day 1.2: {}".format(manhattan_distance((0, 0), next(solve()))))


Day 1.2: 133


## Day 2: Bathroom security
You arrive at **Easter Bunny Headquarters** under cover of darkness. However, you left in such a rush that you forgot to use the bathroom! Fancy office buildings like this one usually have keypad locks on their bathrooms, so you search the front desk for the code.

"In order to improve security," the document you find says, "bathroom codes will no longer be written down. Instead, please memorize and follow the procedure below to access the bathrooms."

The document goes on to explain that each button to be pressed can be found by starting on the previous button and moving to adjacent buttons on the keypad: U moves up, D moves down, L moves left, and R moves right. Each line of instructions corresponds to one button, starting at the previous button (or, for the first line, **the "5" button**); press whatever button you're on at the end of each line. If a move doesn't lead to a button, ignore it.

You can't hold it much longer, so you decide to figure out the code as you walk to the bathroom. You picture a keypad like this:

```
1 2 3
4 5 6
7 8 9
```
Suppose your instructions are:

```
ULL
RRDDD
LURDL
UUUUD
```

- You start at "5" and move up (to "2"), left (to "1"), and left (you can't, and stay on "1"), so the first button is `1`.
- Starting from the previous button ("1"), you move right twice (to "3") and then down three times (stopping at "9" after two moves and ignoring the third), ending up with `9`.
- Continuing from "9", you move left, up, right, down, and left, ending with `8`.
- Finally, you move up four times (stopping at "2"), then down once, ending with `5`.
So, in this example, the bathroom code is `1985`.

Your puzzle input is the instructions from the document you found at the front desk. What is the `bathroom code`?

In [54]:
keypad = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
moves = {
    'U': lambda k: (k[0], max(0, k[1] - 1)),
    'D': lambda k: (k[0], min(2, k[1] + 1)),
    'L': lambda k: (max(0, k[0] - 1), k[1]),
    'R': lambda k: (min(2, k[0] + 1), k[1])
}

with Input(2) as f:
    key = (1, 1)
    sequence = []
    for line in f:
        for c in line.strip():
            key = moves[c](key)
        sequence.append(keypad[key[1]][key[0]])
    print("Day 2.1: {}".format(''.join(map(str, sequence))))

Day 2.1: 78293


### Part 2
You finally arrive at the bathroom (it's a several minute walk from the lobby so visitors can behold the many fancy conference rooms and water coolers on this floor) and go to punch in the code. Much to your bladder's dismay, the keypad is not at all like you imagined it. Instead, you are confronted with the result of hundreds of man-hours of bathroom-keypad-design meetings:

```
    1
  2 3 4
5 6 7 8 9
  A B C
    D
```
You still start at "5" and stop when you're at an edge, but given the same instructions as above, the outcome is very different:

- You start at "5" and don't move at all (up and left are both edges), ending at `5`.
- Continuing from "5", you move right twice and down three times (through "6", "7", "B", "D", "D"), ending at `D`.
- Then, from "D", you move five more times (through "D", "B", "C", "C", "B"), ending at `B`.
- Finally, after five more moves, you end at `3`.
So, given the actual keypad layout, the code would be `5DB3`.

Using the same instructions in your puzzle input, what is the correct **bathroom code**?

In [55]:
keypad = {
    '1': {'D': '3'},
    '2': {'R': '3', 'D': '6'},
    '3': {'U': '1', 'L': '2', 'D': '7', 'R': '4'},
    '4': {'L': '3', 'D': '8'},
    '5': {'R': '6'},
    '6': {'L': '5', 'U': '2', 'R': '7', 'D': 'A'},
    '7': {'L': '6', 'U': '3', 'R': '8', 'D': 'B'},
    '8': {'L': '7', 'U': '4', 'R': '9', 'D': 'C'},
    '9': {'L': '8'},
    'A': {'U': '6', 'R': 'B'},
    'B': {'L': 'A', 'U': '7', 'R': 'C', 'D': 'D'},
    'C': {'U': '8', 'L': 'B'},
    'D': {'U': 'A'}
}

with Input(2) as f:
    key = '5'
    sequence = []
    for line in f:
        for c in line.strip():
            key = keypad[key][c] if c in keypad[key] else key
        sequence.append(key)
    print("Day 2.2: {}".format(''.join(sequence)))

Day 2.2: AC8C8


## Day 3: Squares With Three Sides
Now that you can think clearly, you move deeper into the labyrinth of hallways and office furniture that makes up this part of Easter Bunny HQ. This must be a graphic design department; the walls are covered in specifications for triangles.

Or are they?

The design document gives the side lengths of each triangle it describes, but... `5 10 25`? Some of these aren't triangles. You can't help but mark the impossible ones.

In a valid triangle, the sum of any two sides must be larger than the remaining side. For example, the "triangle" given above is impossible, because `5 + 10` is not larger than `25`.

In your puzzle input, **how many** of the listed triangles are **possible**?

In [56]:
def is_triangle(triangle):
    x, y, z = sorted(triangle)
    return x + y > z

with Input(3) as f:
    triangles = [[int(c) for c in line.strip().split()] for line in f]
    print("Day 3.1: {}".format(sum(map(is_triangle, triangles))))

Day 3.1: 869


Now that you've helpfully marked up their design documents, it occurs to you that triangles are specified in groups of three **vertically**. Each set of three numbers in a column specifies a triangle. Rows are unrelated.

For example, given the following specification, numbers with the same hundreds digit would be part of the same triangle:

```
101 301 501
102 302 502
103 303 503
201 401 601
202 402 602
203 403 603
```
In your puzzle input, and instead reading by columns, **how many** of the listed triangles are **possible**?

In [57]:
triangles = np.transpose(np.array(triangles)).reshape(len(triangles), 3)
print("Day 3.2: {}".format(sum(map(is_triangle, triangles))))

Day 3.2: 1544


## Day 4: Security Through Obscurity
Finally, you come across an information kiosk with a list of rooms. Of course, the list is encrypted and full of decoy data, but the instructions to decode the list are barely hidden nearby. Better remove the decoy data first.

Each room consists of an encrypted name (lowercase letters separated by dashes) followed by a dash, a sector ID, and a checksum in square brackets.

A room is real (not a decoy) if the checksum is the five most common letters in the encrypted name, in order, with ties broken by alphabetization. For example:

- `aaaaa-bbb-z-y-x-123[abxyz]` is a real room because the most common letters are `a` (5), `b` (3), and then a tie between `x`, `y`, and `z`, which are listed alphabetically.
- `a-b-c-d-e-f-g-h-987[abcde]` is a real room because although the letters are all tied (1 of each), the first five are listed alphabetically.
- `not-a-real-room-404[oarel]` is a real room.
- `totally-real-room-200[decoy]` is not.

Of the real rooms from the list above, the sum of their sector IDs is `1514`.

What is the **sum of the sector IDs of the real rooms**?

In [81]:
room_re = re.compile("(?P<name>[a-z-]+)-(?P<sector>[0-9]+)\[(?P<checksum>[a-z]{5})\]")

def real_room(room):
    counter = Counter(room['name'])
    del counter['-']
    top5 = sorted([(-n, c) for c, n in counter.most_common()])[:5]
    top = ''.join([c for n, c in top5])
    return room['checksum'] == top

with Input(4) as f:
    rooms = list(filter(real_room, [room_re.search(r).groupdict() for r in f]))
    print("Day 4.1: {}".format(sum(map(lambda r: int(r['sector']), rooms))))

Day 4.1: 409147


### Part 2
With all the decoy data out of the way, it's time to decrypt this list and get moving.

The room names are encrypted by a state-of-the-art shift cipher, which is nearly unbreakable without the right software. However, the information kiosk designers at Easter Bunny HQ were not expecting to deal with a master cryptographer like yourself.

To decrypt a room name, rotate each letter forward through the alphabet a number of times equal to the room's sector ID. `A` becomes `B`, `B` becomes `C`, `Z` becomes `A`, and so on. Dashes become spaces.

For example, the real name for `qzmt-zixmtkozy-ivhz-343` is `very encrypted name`.

**What is the sector ID** of the room where North Pole objects are stored?

In [85]:
def solve(rooms):
    for room in rooms:
        name, sector = room['name'], int(room['sector'])
        decrypted = ''.join([' ' if c == '-' else chr((((ord(c) - 97) + sector) % 26) + 97) for c in name])
        if 'northpole' in decrypted:
            yield sector

print("Day 4.2: {}".format(next(solve(rooms))))

Day 4.2: 991
